In [1]:
# default_exp modules

This contains pytorch modules which allow for optimization of intrinsic/extrinsic parameters

# Import

In [2]:
# export
import torch
import torch.nn as nn

from camera_calib_python.utils import *

In [3]:
from IPython.core.debugger import set_trace

# Utility

In [4]:
# export
def tensors2parameters(*args): 
    params = [nn.Parameter(arg) for arg in args]
    return params[0] if len(params) == 1 else params

# Inversible Modules

Some things:
* I want to be able to have a `Module`, call `Inverse()` on it, and have the `Module` and it's `Inverse` share the same parameters, yet the inverse does the inverse of the module
* I want to ensure some intuitive things, like: `Inverse(Inverse(Module)) == Module`
* It's useful to be able to access some parameters in an "inverse-aware" reconstructed form, so I've implemented `get_param()`. i.e. if a `Rotation` is `Inverse`d and I want to get the rotation matrix, calling the `get_param()` method should return the transposed version

For inversible modules, inherit this:

In [5]:
# export
class Inversible(nn.Module):
    def __init__(self):
        super().__init__()
       
    def forward_param(self): raise NotImplementedError('Please implement forward_param() method')
    def inverse_param(self): raise NotImplementedError('Please implement inverse_param() method')
        
    # Call this method to get a copy of "inverse aware" reconstructed parameters
    def get_param(self):
        with torch.no_grad():
            return self.forward_param().detach().clone() 
    
    def inverse(self): raise NotImplementedError('Please implement inverse() method')

Do `Inverse(Module)` to take its inverse:

In [6]:
# export
class Inverse(Inversible): # Note that Inverse is itself Inversible...
    def __init__(self, m):
        super().__init__()
        self.m = m
        
    def __repr__(self):
        return f'{self.__class__.__name__}({self.m.__repr__()})'
    
    def forward_param(self): return self.m.inverse_param()
    def inverse_param(self): return self.m.forward_param()
    
    def forward(self, x): return self.m.inverse(x)
    def inverse(self, x): return self.m.forward(x)

Have a basic test to test inversibility

In [7]:
# export
def assert_inversible(m, x, y, **kwargs):
    assert_allclose(m(x),          y, **kwargs)
    assert_allclose(Inverse(m)(y), x, **kwargs)

In [8]:
a = Inversible(); b = Inverse(a); c = Inverse(b)
a,b,c

(Inversible(), Inverse(Inversible()), Inverse(Inverse(Inversible())))

# Translation module

This performs a simple 3D translation; the inverse is simply subtracting the translation

In [9]:
# export
class Translation(Inversible):
    def __init__(self, t):
        super().__init__()
        self.t = tensors2parameters(t)
        
    def __repr__(self):
        return f'{self.__class__.__name__}(tx:{self.t[0]:.4} ty:{self.t[1]:.4} tz:{self.t[2]:.4})'
        
    def forward_param(self): return  self.t
    def inverse_param(self): return -self.forward_param()
        
    def forward(self, ps, inverse=False): 
        t = self.forward_param() if not inverse else self.inverse_param()
        return ps + t
    def inverse(self, ps): return self.forward(ps, inverse=True)

In [10]:
t = torch.Tensor([1,2,3])
a = Translation(t); b = Inverse(a);
a, b

(Translation(tx:1.0 ty:2.0 tz:3.0), Inverse(Translation(tx:1.0 ty:2.0 tz:3.0)))

In [11]:
ps = torch.rand(4,3)
ps

tensor([[0.2641, 0.3741, 0.1961],
        [0.8423, 0.0054, 0.7646],
        [0.8448, 0.8015, 0.4630],
        [0.6405, 0.3620, 0.8003]])

In [12]:
assert_inversible(a, ps, ps+t, atol=1e-4)
assert_inversible(b, ps, ps-t, atol=1e-4)

In [13]:
assert_allclose(a.get_param(),  t)
assert_allclose(b.get_param(), -t)

# Rotation module

This performs a simple 3D rotation; the inverse is simply the transpose. Implement the reconstruction in the `r2R()` method for child classes.

In [14]:
# export
class Rotation(Inversible):
    def __init__(self):
        super().__init__()
        
    def r2R(self):
        raise NotImplementedError('Please implement rotation matrix reconstruction')       
    
    def forward_param(self): return self.r2R()
    def inverse_param(self): return self.forward_param().T
        
    def forward(self, ps, inverse=False): 
        R = self.forward_param() if not inverse else self.inverse_param()
        return pmm(R, ps)
    def inverse(self, ps): return self.forward(ps, inverse=True)

### Euler

Note that in particular, `euler2R` must be differentiable

In [15]:
# export
def R2euler(R):
    atan2 = torch.atan2
    
    return (atan2(R[2, 1], R[2, 2]),
            atan2(-R[2, 0], torch.sqrt(R[0, 0]**2+R[1, 0]**2)),
            atan2(R[1, 0], R[0, 0]))

In [16]:
# export
def euler2R(e_x, e_y, e_z):
    s, c = torch.sin, torch.cos
    stack = torch.stack
    
    return stack([
        stack([c(e_y)*c(e_z), c(e_z)*s(e_x)*s(e_y) - c(e_x)*s(e_z), s(e_x)*s(e_z) + c(e_x)*c(e_z)*s(e_y)]),
        stack([c(e_y)*s(e_z), c(e_x)*c(e_z) + s(e_x)*s(e_y)*s(e_z), c(e_x)*s(e_y)*s(e_z) - c(e_z)*s(e_x)]),
        stack([      -s(e_y),                        c(e_y)*s(e_x),                        c(e_x)*c(e_y)])
    ])

Test to make sure r->R->r is consistent

In [17]:
euler = tuple(torch.rand(3))
euler

(tensor(0.0272), tensor(0.3686), tensor(0.1606))

In [18]:
assert_allclose(euler, R2euler(euler2R(*euler)))

In [19]:
# export
class EulerRotation(Rotation):
    def __init__(self, R):
        super().__init__()
        self.ex, self.ey, self.ez = tensors2parameters(*R2euler(R))
    
    def __repr__(self):
        return f'{self.__class__.__name__}(ex:{self.ex:.4} ey:{self.ey:.4} ez:{self.ez:.4})'
    
    def r2R(self): return euler2R(self.ex, self.ey, self.ez)

In [20]:
R = euler2R(*torch.Tensor([1,2,3]))
a = EulerRotation(R); b = Inverse(a)
a,b

(EulerRotation(ex:-2.142 ey:1.142 ez:-0.1416),
 Inverse(EulerRotation(ex:-2.142 ey:1.142 ez:-0.1416)))

In [21]:
ps = torch.rand(4,3)
ps

tensor([[0.7095, 0.3799, 0.7693],
        [0.6361, 0.9784, 0.2586],
        [0.8211, 0.3080, 0.9836],
        [0.7272, 0.8484, 0.0771]])

In [22]:
assert_inversible(a, ps, pmm(R,   ps), atol=1e-4)
assert_inversible(b, ps, pmm(R.T, ps), atol=1e-4)

In [23]:
assert_allclose(a.get_param(), R)
assert_allclose(b.get_param(), R.T)

# InversibleSequential Module

Having inversible composible modules will be super useful. This is basically nn.Sequential but I've made it inversible. Note that:
`y = f(g(h(x)))` implies that `x = h^-1(g^-1(f^-1(y)))`. So basically just reverse the order and take the inverse of each function to invert the whole thing.

In [24]:
# export
class InversibleSequential(Inversible):
    def __init__(self, ms):
        super().__init__()
        self.ms = nn.ModuleList(ms)  
    
    def forward(self, x):
        for m in self.ms: 
            x = m.forward(x)
        return x
    
    def inverse(self, x):
        for m in reversed(self.ms):
            x = m.inverse(x)
        return x

# Rigid

Rigid transform is a rotation followed by translation.

In [25]:
# export
def Rt2M(R, t):
    M = torch.cat([R, t[:,None]], dim=1)
    M = torch.cat([M, M.new_tensor([[0,0,0,1]])])
    return M

In [26]:
# export
def M2Rt(M): return M[0:3,0:3], M[0:3,3]

In [27]:
# export
def invert_rigid(M):
    R, t = M2Rt(M)
    return Rt2M(R.T, -R.T@t)

Test to make sure `torch.inverse` returns same answer as `invert_rigid` for rigid transformations

In [28]:
R, t = euler2R(*tuple(torch.Tensor([1,2,3]))), torch.Tensor([1,2,3])
M = Rt2M(R, t)
assert_allclose(invert_rigid(M), torch.inverse(M))

In [29]:
# export
class Rigid(InversibleSequential):
    def __init__(self, R, t, Rotation=EulerRotation):
        super().__init__([Rotation(R), Translation(t)]) # NOTE: rotation happens first
        
    def forward_param(self):
        return Rt2M(self.ms[0].forward_param(), # R 
                    self.ms[1].forward_param()) # t
        
    def inverse_param(self): return invert_rigid(self.forward_param())

In [30]:
R, t = euler2R(*tuple(torch.Tensor([1,2,3]))), torch.Tensor([1,2,3])
a = Rigid(R, t); b = Inverse(a)
a,b

(Rigid(
   (ms): ModuleList(
     (0): EulerRotation(ex:-2.142 ey:1.142 ez:-0.1416)
     (1): Translation(tx:1.0 ty:2.0 tz:3.0)
   )
 ), Inverse(Rigid(
   (ms): ModuleList(
     (0): EulerRotation(ex:-2.142 ey:1.142 ez:-0.1416)
     (1): Translation(tx:1.0 ty:2.0 tz:3.0)
   )
 )))

In [31]:
ps = torch.rand(4,3)
ps

tensor([[0.0296, 0.7165, 0.0081],
        [0.6325, 0.6566, 0.4154],
        [0.1827, 0.6999, 0.9354],
        [0.4255, 0.9967, 0.7981]])

In [32]:
M = Rt2M(R,t)
M_inv = invert_rigid(M)
R_inv, t_inv = M2Rt(M_inv)

In [33]:
assert_inversible(a, ps, pmm(R,     ps) + t,     atol=1e-4)
assert_inversible(b, ps, pmm(R_inv, ps) + t_inv, atol=1e-4)

In [34]:
assert_allclose(a.get_param(), M)
assert_allclose(b.get_param(), M_inv)

Create a wrapper for composing multiple `Rigid`s

In [35]:
# export
def mult_rigid(M1, M2):
    R1, t1 = M2Rt(M1)
    R2, t2 = M2Rt(M2)
    return Rt2M(R1@R2, R1@t2+t1)

In [36]:
R1, t1 = euler2R(*tuple(torch.Tensor([1,2,3]))), torch.Tensor([1,2,3])
R2, t2 = euler2R(*tuple(torch.Tensor([2,2,2]))), torch.Tensor([2,2,2])
M1, M2 = Rt2M(R1, t1), Rt2M(R2, t2)
assert_allclose(mult_rigid(M2, M1), M2@M1)

In [37]:
# export
class Rigids(InversibleSequential):
    def __init__(self, rigids):
        super().__init__(rigids)
    
    def forward_param(self):
        M = torch.eye(4)
        for m in self.ms:
            M = mult_rigid(m.forward_param(), M)
        return M
    
    def inverse_param(self): return invert_rigid(self.forward_param())

In [38]:
R_a = euler2R(*tuple(torch.Tensor([ 1, 2, 3]))); t_a = torch.Tensor([1,2,3]); a = Rigid(R_a, t_a)
R_b = euler2R(*tuple(torch.Tensor([.5, 1, 1]))); t_b = torch.Tensor([3,2,2]); b = Inverse(Rigid(R_b, t_b))
R_c = euler2R(*tuple(torch.Tensor([.3,.1, 0]))); t_c = torch.Tensor([3,3,3]); c = Rigid(R_c, t_c)

In [39]:
d = Rigids([a,b,c])

In [40]:
M_a = Rt2M(R_a, t_a)
M_b = Rt2M(R_b.T, -R_b.T@t_b)
M_c = Rt2M(R_c, t_c)
M_d = M_c@M_b@M_a

In [41]:
R_d, t_d = M2Rt(M_d)

In [42]:
assert_inversible(d, ps, pmm(R_d, ps) + t_d, atol=1e-4)

In [43]:
assert_allclose(d.get_param(), M_d)

# Normalize

This will normalize 3D coordinates (i.e. project points to "unit" image plane).

In [44]:
# export
class Normalize(nn.Module): # Note: Not Inversible
    def __init__(self):
        super().__init__()
        
    def forward(self, ps): return normalize(ps)

In [45]:
ps = torch.rand(4,3)
ps

tensor([[0.9246, 0.9636, 0.9497],
        [0.1860, 0.3661, 0.5820],
        [0.7747, 0.5847, 0.6103],
        [0.6113, 0.9318, 0.2500]])

In [46]:
assert_allclose(Normalize()(ps), ps[:,0:2]/ps[:,2:3])

# Augment

Adds `one`s to input array; note this is `Inversible`

In [47]:
# export
class Augment(Inversible):
    def __init__(self):
        super().__init__()

    def forward(self, ps): return augment(ps)
    def inverse(self, ps): return deaugment(ps)

Test to make sure `one`s are added and removed

In [48]:
ps = torch.rand(4,2)
ps

tensor([[0.2071, 0.1933],
        [0.9710, 0.6452],
        [0.6830, 0.1576],
        [0.7037, 0.2313]])

In [49]:
assert_inversible(Augment(), ps, torch.cat([ps, torch.ones(len(ps),1)], dim=1))

# Distortion

This will apply distortion model to 2D points. Note for now I didn't find much commonality so I didn't make a base class. Also note that distortions are typically not inversible (at least not analytically to my knowledge), so exclude for now.

### No distortion

In [50]:
# export
class NoDistortion(nn.Module):
    def __init__():
        super().__init__()
        
    def forward(self, ps): return ps

### Heikkila97 distortion

In [51]:
# export
class Heikkila97Distortion(nn.Module):
    def __init__(self, k1, k2, p1, p2):
        super().__init__()
        self.k1, self.k2, self.p1, self.p2 = tensors2parameters(k1, k2, p1, p2)
    
    def __repr__(self):
        return f'{self.__class__.__name__}(k1:{self.k1:.4} k2:{self.k2:.4} p1:{self.p1:.4} p2:{self.p2:.4})'
    
    def forward(self, ps):
        xs, ys = ps[:,0], ps[:,1]
        
        # Radial distortion
        k1, k2 = self.k1, self.k2
        rs = xs**2 + ys**2
        xs_r = xs*(1 + k1*rs + k2*rs**2)
        ys_r = ys*(1 + k1*rs + k2*rs**2)

        # Decentering distortion
        p1, p2 = self.p1, self.p2        
        xs_d = xs_r + 2*p1*xs*ys + p2*(3*xs**2 + ys**2)
        ys_d = ys_r + p1*(xs**2 + 3*ys**2) + 2*p2*xs*ys
        
        return torch.stack([xs_d, ys_d]).T

In [52]:
distortion = Heikkila97Distortion(*torch.Tensor([.01, 0.02, 0.03, 0.04]))
distortion

Heikkila97Distortion(k1:0.01 k2:0.02 p1:0.03 p2:0.04)

In [53]:
ps = torch.Tensor([[0.1900, 0.2467],
                   [0.9817, 0.7349],
                   [0.2432, 1.6161],
                   [0.2194, 0.4353]])
assert_allclose(distortion(ps),
                torch.Tensor([[0.1998, 0.2573],
                              [1.2215, 0.9145],
                              [0.4196, 2.1581],
                              [0.2393, 0.4630]]), atol=1e-4)

### Wang08 distortion

In [54]:
# export
class Wang08Distortion(nn.Module):
    def __init__(self, k1, k2, p, t):
        super().__init__()
        self.k1, self.k2, self.p, self.t = tensors2parameters(k1, k2, p, t)
    
    def __repr__(self):
        return f'{self.__class__.__name__}(k1:{self.k1:.4} k2:{self.k2:.4} p:{self.p:.4} t:{self.t:.4})'
    
    def forward(self, ps):
        xs, ys = ps[:,0], ps[:,1]
        
        # Radial distortion
        k1, k2 = self.k1, self.k2
        rs = xs**2 + ys**2
        xs_r = xs*(1 + k1*rs + k2*rs**2)
        ys_r = ys*(1 + k1*rs + k2*rs**2)

        # Image plane (small angle approximation) rotation distortion
        p, t = self.p, self.t
        xs_d = xs_r/(-p*xs_r + t*ys_r + 1)
        ys_d = ys_r/(-p*xs_r + t*ys_r + 1)
        
        return torch.stack([xs_d, ys_d]).T

NOTE: possibly refactor the radial distortion part since it appears twice

In [55]:
distortion = Wang08Distortion(*torch.Tensor([.01, 0.02, 0.03, 0.04]))
distortion

Wang08Distortion(k1:0.01 k2:0.02 p:0.03 t:0.04)

In [56]:
ps = torch.Tensor([[0.1900, 0.2467],
                   [0.9817, 0.7349],
                   [0.2432, 1.6161],
                   [0.2194, 0.4353]])
assert_allclose(distortion(ps),
                torch.Tensor([[0.1894, 0.2460],
                              [1.0409, 0.7792],
                              [0.2665, 1.7711],
                              [0.2178, 0.4321]]), atol=1e-4)

# Camera matrix

This performs the application of the Camera matrix. Implement the reconstruction in the `a2A()` method for child classes.

In [57]:
# export
def a2A(alpha_x, alpha_y, x_o, y_o):
    stack = torch.stack
    zero, one = alpha_x.new_zeros(1)[0], alpha_x.new_ones(1)[0]
    return stack([
        stack([alpha_x,    zero, x_o]),
        stack([   zero, alpha_y, y_o]),
        stack([   zero,    zero, one])
    ])

NOTE: Possibly refactor `stack(stack())` since I've used this a couple times

Test to make sure reconstruction is correct

In [58]:
a = torch.Tensor([1.5, -2.5, 3.5])
A = a2A(alpha_x=a[0], alpha_y=a[0], x_o=a[1], y_o=a[2])
assert_allclose(A, torch.Tensor([[ 1.5,  0.0, -2.5],
                                 [ 0.0,  1.5,  3.5],
                                 [ 0.0,  0.0,  1.0]]))

In [59]:
# export
class Cam(Inversible):
    def __init__(self):
        super().__init__()
    
    def a2A(self):
        raise NotImplementedError('Please implement camera matrix reconstruction')
    
    def forward_param(self): return self.a2A()
    def inverse_param(self): return torch.inverse(self.forward_param()) # Overwrite this in child classes
        
    def forward(self, ps, inverse=False):
        A = self.forward_param() if not inverse else self.inverse_param()
        return pmm(A, ps, aug=True)
    def inverse(self, ps): return self.forward(ps, inverse=True)

### Single focal length

Assumes there is only a single `alpha`

In [60]:
# export
class CamSF(Cam):
    def __init__(self, A):
        super().__init__()
        assert_allclose(A[0,0], A[1,1]) # Check that alpha_x and alpha_y are the same
        self.alpha, self.x_o, self.y_o = tensors2parameters(A[0,0], A[0,2], A[1,2])
    
    def __repr__(self):
        return f'{self.__class__.__name__}(alpha:{self.alpha:.4} x_o:{self.x_o:.4} y_o:{self.y_o:.4})'
    
    def a2A(self):
        return a2A(alpha_x=self.alpha, alpha_y=self.alpha, x_o=self.x_o, y_o=self.y_o)
    
    def inverse_param(self): # Overwrite parent method
        return a2A(alpha_x=1/self.alpha, 
                   alpha_y=1/self.alpha, 
                   x_o=-self.x_o/self.alpha, 
                   y_o=-self.y_o/self.alpha)

In [61]:
a = torch.Tensor([1.5, -2.5, 3.5])
A = a2A(alpha_x=a[0], alpha_y=a[0], x_o=a[1], y_o=a[2])
cam = CamSF(A)
cam

CamSF(alpha:1.5 x_o:-2.5 y_o:3.5)

Test

In [62]:
ps = torch.rand(4,2)
ps

tensor([[0.4940, 0.8915],
        [0.6462, 0.3791],
        [0.4745, 0.3810],
        [0.3633, 0.6383]])

In [63]:
assert_inversible(cam, ps, pmm(A, ps, aug=True), atol=1e-4)

In [64]:
assert_allclose(cam.get_param(), A)

# Build

In [65]:
!nbdev_build_lib

Converted calib.ipynb.
Converted cb_geom.ipynb.
Converted control_refine.ipynb.
Converted fiducial_detect.ipynb.
Converted image.ipynb.
Converted modules.ipynb.
Converted utils.ipynb.
